# [Лаба 3 - Временные ряды](https://www.kaggle.com/competitions/mai-ml-contest-time-series/overview)
Данное соревнование проводится в рамках курса по Машинному Обучению в МАИ для 3го курса бакалавриата 2024 года.

Цель: развить навыки анализа, подготовки данных и построения моделей решающих задачу линейной регрессии по набору данных.

Для успешного прохождения соревнования в рамках лабораторной работы необходимо получить значение MAPE менее чем 0.12.

В коментарии к submit'у вам нужно приложить ссылку на ноутбук которым был получен данный сабмит

[диск с инфой](https://disk.yandex.ru/d/wYViCThVfBe-ZA)

# Правила:
- Разрешено использование сколь угодно сложных моделей, кроме древесных (бустинг, деревья, леса). В том числе для подготовки данных.
- Запрещено использовать результаты работы других студентов за исключением обмена опытом.
- Полное совпадение метрик приведет к пересмотру ваших работ. В случае если выяснится полное заимствование - дисквалификация с соревнований и незачет по лабораторной.

# Состав ноутбука:
- Блок загрузки данных
- Блок анализа данных
- Блок подготовки и очистки данных
- Блок обучения и тестирования моделей
- Блок сабмита

# Задача
Классическая задача прогноза продаж в сети магазинов.

Разные магазины в зависимости от различных факторов могут иметь различные продажи. Чтобы понять сколько товара нужно поставить в магазин, какая ожидается выручка и т.п. нам необходимо уметь предсказывать продажи в магазинах в зависимости от региона, сезона, температуры на улице, стоимости топлива, экономической ситуации и т.д.

Ваша задача предсказать продажи магазина на месяц вперед (4 недели) на основе истории.

## Входные данные
- Файлы
  - train.csv - тренировочный набор данных. С этим набором вы делаете все что хотите, там есть true значения и все фичи.
  - test.csv - тестовый набор данных. Здесь вам истинные значения будут не известны. Вы лишь можете сделать сабмит с помощью этого набора.

Описание столбцов
| название | описание |
|-|-|
| Store | ID магазина |
| Date | дата на момент фиксации фич |
| Weekly_Sales | продажи за неделю |
| Temperature | средняя температура за неделю в городе расположения магазина (в градусах Фарингейта) |
| Fuel_Price | цена топлива в данном регионе |
| CPI | индекс потребительских цен |
| Unemployment | уровень безработицы в регионе |

Ваша цель - предсказать Weekly_Sales на 4 измерения вперед (на месяц).

# Поехали

In [1]:
import pandas as pd

import numpy as np

from scipy.optimize import minimize
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
train_data_path = 'train.csv'
test_data_path = 'test.csv'

In [3]:
initial_df = pd.read_csv(train_data_path, parse_dates=['Date'])
initial_df.describe()

,Store,Date,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment
count,6255.000000,6255,6.255000e+03,6255.000000,6255.000000,6255.000000,6255.000000
mean,23.000000,2011-06-03 05:21:09.064748032,1.047722e+06,60.647365,3.344193,171.439895,8.025884
min,1.000000,2010-01-10 00:00:00,2.099862e+05,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,2010-10-09 00:00:00,5.538695e+05,47.170000,2.917000,131.735000,6.908000
50%,23.000000,2011-05-27 00:00:00,9.609985e+05,62.730000,3.413000,182.610406,7.906000
75%,34.000000,2012-02-03 00:00:00,1.422573e+06,75.210000,3.722000,212.488702,8.622000
max,45.000000,2012-10-08 00:00:00,3.818686e+06,100.140000,4.308000,226.966232,14.313000
std,12.988211,NaN,5.654876e+05,18.623215,0.455260,39.297817,1.875644


# Посмотрим на основные распределения в изначальном датасете

In [4]:
go.Figure(
    go.Scatter(
        x=initial_df['Date'],
        y=initial_df.groupby('Date').agg('sum')['Weekly_Sales'],
    ),
    layout=dict(
        title='Суммарные продажи за неделю',
        width=1000,
        height=500,
    )
).show()


per_store_df = { store: data for store, data in initial_df.groupby('Store') }

all_per_store = make_subplots(
    rows=5, cols=1,
    subplot_titles=[
        'Недельные продажи',
        'Средняя температура, °F',
        'Цена на топливо',
        'Индекс потребительских цен',
        'Уровень безработицы',
    ]
)

for store, submit in per_store_df.items():
    all_per_store.add_trace(
        go.Scatter(
            x=submit['Date'],
            y=submit['Weekly_Sales'],
            mode='lines',
            name=f'Магазин №{store}',
            opacity=0.1,
        ),
        row=1, col=1
    )

    all_per_store.add_trace(
        go.Scatter(
            x=submit['Date'],
            y=submit['Temperature'],
            mode='lines',
            name=f'Магазин №{store}',
            opacity=0.1,
        ),
        row=2, col=1
    )

    all_per_store.add_trace(
        go.Scatter(
            x=submit['Date'],
            y=submit['Fuel_Price'],
            mode='lines',
            name=f'Магазин №{store}',
            opacity=0.1,
        ),
        row=3, col=1
    )

    all_per_store.add_trace(
        go.Scatter(
            x=submit['Date'],
            y=submit['CPI'],
            mode='lines',
            name=f'Магазин №{store}',
            opacity=0.1,
        ),
        row=4, col=1
    )

    all_per_store.add_trace(
        go.Scatter(
            x=submit['Date'],
            y=submit['Unemployment'],
            mode='lines',
            name=f'Магазин №{store}',
            opacity=0.1,
        ),
        row=5, col=1
    )

all_per_store.update_layout(
    title='Значения в разных магазинах',
    showlegend=False,
    width=1000,
    height=1500,
)

all_per_store.show()

go.Figure(
    go.Splom(
        dimensions=[
            dict(name='Weekly_Sales', label='Weekly_Sales', values=initial_df[initial_df['Store'] == 1]['Weekly_Sales']),
            dict(name='Temperature', label='Temperature', values=initial_df[initial_df['Store'] == 1]['Temperature']),
            dict(name='Fuel_Price', label='Fuel_Price', values=initial_df[initial_df['Store'] == 1]['Fuel_Price']),
            dict(name='CPI', label='CPI', values=initial_df[initial_df['Store'] == 1]['CPI']),
            dict(name='Unemployment', label='Unemployment', values=initial_df[initial_df['Store'] == 1]['Unemployment']),
        ],
        showupperhalf=False,
        marker=dict(size=2),
        diagonal=dict(visible=False)
    ),
    layout=dict(width=1000, height=1000, title='Корреляции фич'),
).show()

На графике видны три пика - в середине октября, конце ноября и конце декабря

От других фич количество продаж никак не зависит

Посмотрим что происходит в скользящих

In [5]:
def plot_moving(series, rolling_data, n, plot_bounds=False, label='Rolling stat'):
    out = []

    out.append(go.Scatter(
        x=series.index[n:],
        y=series.values[n:],
        mode='lines',
        name='Actual values',
        line=dict(color='blue'),
        opacity=0.5,
    ))

    if plot_bounds:
        rolling_std = series.rolling(window=n).std()
        upper_bound = rolling_data + 1.96 * rolling_std
        lower_bound = rolling_data - 1.96 * rolling_std

        out.append(go.Scatter(
            name='Upper Bound',
            x=upper_bound.index,
            y=upper_bound,
            line=dict(color='red', dash='dash'),
        ))

        out.append(go.Scatter(
            name='Lower Bound',
            x=lower_bound.index,
            y=lower_bound,
            line=dict(color='red', dash='dash'),
            fill='tonexty',
            fillcolor='rgba(255, 0, 0, 0.05)',
        ))

    out.append(go.Scatter(
        name=label,
        x=rolling_data.index,
        y=rolling_data,
        line=dict(color='red'),
    ))

    return out

def plot_moving_average(series, n, plot_bounds=False):
    rolling_data = series.rolling(window=n).mean()
    return plot_moving(series, rolling_data, n, plot_bounds, label='Rolling average')


moving_averages = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Сумма, скользящая по месяцу',
        'Медиана, скользящая по месяцу',
        'Сумма, скользящая по сезону',
        'Медиана, скользящая по сезону',
    ]
)

agg_sum_df = initial_df.groupby('Date').agg('sum')
agg_mean_df = initial_df.groupby('Date').median()
plot_bounds = True

for p in plot_moving_average(agg_sum_df['Weekly_Sales'], 4, plot_bounds=plot_bounds):
    moving_averages.add_trace(p, row=1, col=1)

for p in plot_moving_average(agg_mean_df['Weekly_Sales'], 4, plot_bounds=plot_bounds):
    moving_averages.add_trace(p, row=1, col=2)


for p in plot_moving_average(agg_sum_df['Weekly_Sales'], 12, plot_bounds=plot_bounds):
    moving_averages.add_trace(p, row=2, col=1)

for p in plot_moving_average(agg_mean_df['Weekly_Sales'], 12, plot_bounds=plot_bounds):
    moving_averages.add_trace(p, row=2, col=2)

moving_averages.update_layout(
    showlegend=False,
    width=1200,
    height=600,
    template='plotly_white',
)
moving_averages.show()

По графикам явно виднеется, что есть подъем в продажах перед новым годом и просадка сразу после, в другое время особо нет движения, значит есть годовая сезонность

Я думаю из-за большого периода между считыванием показаний, другой сезонности тут нет

# Хольт-Винтер
Построим модель Хольт-Винтера для каждого магазина

In [6]:
class HoltWinters:
    '''
    Модель Хольта-Винтерса с методом Брутлага для детектирования аномалий
    https://fedcsis.org/proceedings/2012/pliks/118.pdf

    # series - исходный временной ряд
    # season_length - длина сезона
    # alpha, beta, gamma - коэффициенты модели Хольта-Винтерса
    # n_preds - горизонт предсказаний
    # scaling_factor - задаёт ширину доверительного интервала по Брутлагу (обычно принимает значения от 2 до 3)
    '''

    def __init__(self, series, season_length, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.season_length = min(season_length, len(self.series))
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor

    def initial_trend(self):
        sum = 0.0
        for i in range(self.season_length):
            if self.season_length + i >= len(self.series): break
            sum += float(self.series[self.season_length + i] - self.series[i]) / self.season_length
        return sum / self.season_length

    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series) / self.season_length)
        # вычисляем сезонные средние
        for j in range(n_seasons):
            season_averages.append(
                sum(
                    self.series[self.season_length * j : self.season_length * j + self.season_length]
                ) / float(self.season_length)
            )
        # вычисляем начальные значения
        for i in range((self.season_length)):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.season_length * j + i] - season_averages[j]
            if n_seasons != 0: seasonals[i] = sum_of_vals_over_avg / n_seasons
        return seasonals

    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBound = []
        self.LowerBound = []

        seasonals = self.initial_seasonal_components()

        for i in range(len(self.series) + self.n_preds):
            if i == 0:  # инициализируем значения компонент
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i % self.season_length])

                self.PredictedDeviation.append(0)

                self.UpperBound.append(
                    self.result[0] + self.scaling_factor * self.PredictedDeviation[0]
                )

                self.LowerBound.append(
                    self.result[0] - self.scaling_factor * self.PredictedDeviation[0]
                )

                continue
            if i >= len(self.series): # прогнозируем
                m = i - len(self.series) + 1
                self.result.append((smooth + m * trend) + seasonals[i % self.season_length])

                # во время прогноза с каждым шагом увеличиваем неопределенность
                self.PredictedDeviation.append(self.PredictedDeviation[-1] * 1.01)

            else:
                val = self.series[i]
                last_smooth, smooth = (
                    smooth,
                    self.alpha * (val - seasonals[i % self.season_length])
                    + (1 - self.alpha) * (smooth + trend),
                )
                trend = self.beta * (smooth - last_smooth) + (1 - self.beta) * trend
                seasonals[i % self.season_length] = (
                    self.gamma * (val - smooth) +
                    (1 - self.gamma) * seasonals[i % self.season_length]
                )
                self.result.append(smooth + trend + seasonals[i % self.season_length])

                # Отклонение рассчитывается в соответствии с алгоритмом Брутлага
                self.PredictedDeviation.append(
                    self.gamma * np.abs(self.series[i] - self.result[i]) +
                    (1 - self.gamma) * self.PredictedDeviation[-1]
                )

            self.UpperBound.append(
                self.result[-1] + self.scaling_factor * self.PredictedDeviation[-1]
            )

            self.LowerBound.append(
                self.result[-1] - self.scaling_factor * self.PredictedDeviation[-1]
            )

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i % self.season_length])

In [7]:
def timeseriesCVscore(x, data, splits, season_length):
    alpha, beta, gamma = x
    values = data.values
    tscv = TimeSeriesSplit(n_splits=splits)

    errors = []

    # идем по фолдам, на каждом обучаем модель, строим прогноз на отложенной выборке и считаем ошибку
    for train, test in tscv.split(values):
        model = HoltWinters(
            series=values[train],
            season_length=season_length,
            alpha=alpha,
            beta=beta,
            gamma=gamma,
            n_preds=len(test),
        )

        model.triple_exponential_smoothing()

        predictions = model.result[-len(test):]
        actual = values[test]
        error = mean_absolute_error(predictions, actual)
        errors.append(error)

    return np.mean(np.array(errors))

Тренировка

In [8]:
models = {}
models_parameters = {}
predict_count = 8
season_length = 52

for store_id, submit in list(per_store_df.items()):
    x = [0, 0, 0]
    train_data = submit['Weekly_Sales'][:-6]
    opt = minimize(lambda x: timeseriesCVscore(x, train_data, 10, season_length), x0=x, method='TNC', bounds=((0, 1), (0, 1), (0, 1)))
    models_parameters[store_id] = (alpha, beta, gamma) = opt.x

    test_data = submit['Weekly_Sales'][:-predict_count]
    models[store_id] = HoltWinters(
        test_data.values,
        season_length=season_length,
        alpha=alpha,
        beta=beta,
        gamma=gamma,
        n_preds=predict_count,
        scaling_factor=2.56,
    )
    models[store_id].triple_exponential_smoothing()

Посмотрим на паре моделях, что получается

In [9]:
def plot_holt_winters(model: HoltWinters, data, predict_count):
    anomalies = np.array([np.nan] * len(data))
    anomalies[data.values < model.LowerBound] = data.values[data.values < model.LowerBound]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        name='Confidence Interval',
        x=list(range(len(model.result))) + list(range(len(model.result))[::-1]),
        y=np.concatenate([model.UpperBound, model.LowerBound[::-1]]),
        fillcolor='rgba(255, 0, 0, 1)',
        line=dict(color='red', dash='dash'),
        opacity=0.5,
    ))

    fig.add_trace(go.Scatter(
        name='Model',
        x=list(range(len(model.result))),
        y=model.result,
        line=dict(color='red'),
        opacity=0.5,
    ))

    fig.add_trace(go.Scatter(
        name='Actual',
        x=list(range(len(data))),
        y=data.values,
        line=dict(color='blue'),
        opacity=0.5,
    ))

    # Anomalies
    fig.add_trace(go.Scatter(
        name='Anomalies',
        x=list(range(len(anomalies))),
        y=anomalies,
        mode='markers',
        marker=dict(color='orange', size=5, symbol='circle')
    ))

    fig.add_shape(
        type='rect',
        x0=len(data) - predict_count,
        x1=len(data),
        y0=min(model.LowerBound),
        y1=max(model.UpperBound),
        fillcolor='grey',
        opacity=0.1,
        line=dict(width=0)
    )

    fig.update_layout(
        title='Holt-Winters Forecast with Confidence Intervals and Anomalies',
        xaxis_title='Time',
        yaxis_title='Values',
        width=1000,
        height=500,
        template='plotly_white',
    )

    fig.show()

plot_holt_winters(models[1], per_store_df[1]['Weekly_Sales'], predict_count)

In [10]:
test_df = pd.read_csv(test_data_path, parse_dates=['Date'])
print(initial_df['Date'].max())
dates = test_df['Date'].unique()
print(dates)

2012-10-08 00:00:00
<DatetimeArray>
['2012-10-19 00:00:00', '2012-10-26 00:00:00', '2012-11-05 00:00:00',
 '2012-12-10 00:00:00']
Length: 4, dtype: datetime64[ns]


Даты конечно идут неровно, но мне слишком лень, чтобы что-то придумать и пока что я забрутфорщу это все...

# Submission

In [11]:
submit = pd.DataFrame(columns=['ID', 'Weekly_Sales'])
submit.set_index(['ID'])

dates = test_df['Date'].unique()

for store_id, data in per_store_df.items():
    model = models[store_id]
    model.triple_exponential_smoothing()

    submit.loc[len(submit)] = {'Weekly_Sales': model.result[1], 'ID': len(submit)}
    submit.loc[len(submit)] = {'Weekly_Sales': model.result[2], 'ID': len(submit)}
    submit.loc[len(submit)] = {'Weekly_Sales': model.result[3], 'ID': len(submit)}
    submit.loc[len(submit)] = {'Weekly_Sales': model.result[8], 'ID': len(submit)}

submit.to_csv('submission.csv', index=False)